In [124]:
import matplotlib.pyplot as plt
import random
import numpy as np
import scipy.stats as sts
from scipy.integrate import quad

In [125]:
n=100
teta=3 #некоторое значение параметра 
x=sts.pareto(b=teta).rvs(100)
beta=0.95

In [126]:
# b) Доверительный интервал для медианы

In [127]:
t=n/np.sum(np.log(x))+1 # омп
a=2**(1/(t-1))*(1-1.97/(np.sqrt(n)*(t-1)))
b=2**(1/(t-1))*(1+1.97/(np.sqrt(n)*(t-1)))
print('Доверительный интервал для медианы(', round(a,3), ',', round(b, 3), ')')
x_med=(np.sort(x)[49] + np.sort(x)[48])/2
print("x_med", x_med)

Доверительный интервал для медианы( 1.154 , 1.295 )
x_med 1.2165159383672508


In [128]:
#quantile(x, (1+beta)/2)=1.97  для beta=0.95
#quantile(x, (1+beta)/2)=-1.97 

In [129]:
#c) Байесовский доверительный интервал 

In [130]:
p=np.product(x)

In [131]:
def f(x,n,p):
    return np.exp(1-x)*(x-1)**n / (p**x)

def integral_1(p, c=None, v=0.025):
    integral=0
    low=1
    hight=10e2
    step=10e-4
    mas_index=np.arange(low, hight, step)
    
    if c is not None:
        for i in range(len(mas_index)-1):
                    t=step*0.5*(f(mas_index[i],n,p)+f(mas_index[i+1], n, p))*c
                    integral+=t
                    if integral>=v:
                        return mas_index[i]
    else:
        for i in range(len(mas_index)-1):
                t=step*0.5*(f(mas_index[i], n, p)+f(mas_index[i+1], n, p)) 
                integral+=t
        return integral

In [132]:
integ=integral_1(p)
c=1/integ
a_b=integral_1(p,c,0.025)
b_b=integral_1(p,c,0.975)
print('Байесовский доверительный интервал (', np.round(a_b,4), ',', np.round(b_b,4), ')')
print('teta', teta)

<ipython-input-131-5545b68d24d7>:2: RuntimeWarning: overflow encountered in double_scalars
  return np.exp(1-x)*(x-1)**n / (p**x)


Байесовский доверительный интервал ( 3.719 , 5.019 )
teta 3


In [133]:
#d) Асимпотический доверительный интервал для  параметра teta

In [134]:
a1=t-1.96*(t-1)/(n**0.5)
b1=t+1.96*(t-1)/(n**0.5)
print('Асимпотический доверительный интервал для параметра  teta: (', round(a1,3), ',', round(b1, 3), ')')
print('teta', teta)

Асимпотический доверительный интервал для параметра  teta: ( 3.748 , 5.088 )
teta 3


In [135]:
# e) Вootstrap

In [136]:
#непараметрический 

In [145]:
s_delta=[]
for i in range(1000):
    bootstrap=np.random.choice(x,size=x.shape[0]).round(3)
    delta=n/np.sum(np.log(bootstrap))+1-t
    s_delta.append(delta)
a3=np.sort(np.array(s_delta))[24]+t
b3=np.sort(np.array(s_delta))[974]+t
print('Bootstrap доверительный интервал (', round(a3,3), ', ', round(b3, 3), ')')
print('teta', teta)

Bootstrap доверительный интервал ( 3.886 ,  5.223 )
teta 3


In [143]:
#параметрический

In [148]:
delta_p=[]
N=20000
for i in range(N):
    bootstrap=sts.pareto(b = teta).rvs(100)
    delta=n/np.sum(np.log(bootstrap))+1-t
    delta_p.append(delta)
k1=int((1-beta)/2 * N)
k2=int((1+beta)/2 * N)
a_par=t+np.sort(np.array(delta_p))[k1]
b_par=t+np.sort(np.array(delta_p))[k2]
print('параметрический bootstrap доверительный интервал (', round(a_par,3), ', ', round(b_par, 3), ')') 
print('teta', teta)

параметрический bootstrap доверительный интервал ( 3.491 ,  4.695 )
teta 3
